In [56]:
using LinearAlgebra
using Random
using Plots
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots C:\Users\Dylan\.julia\packages\Plots\M1wcx\src\backends.jl:373


Plots.PlotlyBackend()

## Erreur inverse - Cas symétrique


Soient $A\in \mathcal{S}_n(\mathbb{R})$ inversible, $b\in\mathbb{R}^n$ et $x\in\mathbb{R}^n$ solution de $$Ax=b.$$

On dispose d'un algorithme permettant la résolution numérique d'un tel problème. L'exécution de celui-ci conduit à une solution numérique $\tilde{x}$, entachée d'erreurs : $$A\tilde{x}\neq b. $$

On considère par contre $\tilde{x}$ la solution d'un autre système linéaire : $$(A+\Delta)\tilde{x} = b. $$

L'erreur inverse de cet algorithme, en norme de Frobenius, peut être obtenue depuis la solution du problème d'optimisation suivant : $$\min_{\Delta \in \mathcal{C}} \Vert \Delta \Vert^2_F,$$

avec $$\mathcal{C}=\left\{ \Delta \in \mathcal{S}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b\right\}.$$

On s'intéresse à un problème plus simple qui consiste à chercher un élément de $\mathcal{C}$, si ce dernier n'est pas vide :

$$(\mathcal{E})\quad \mbox{ Trouver }\Delta \in \mathcal{S}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b.$$

En écrivant $$\mathcal{C}=\mathcal{C}_1\bigcap\mathcal{C}_2,$$ avec $$\mathcal{C}_1=\mathcal{S}_n(\mathbb{R}), \quad \mathcal{C}_2=\left\{\Delta \in \mathcal{M}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b\right\},$$ on est amenés à chercher un élément sur l'intersection de deux convexes fermés non vides.

**Question 1 :** Justifier que ces deux parties sont convexes, fermées et non vides.

On définit pour cela le problème d'optimisation suivant : 
$$(\mathcal{P})\quad \min_{\Delta \in \mathcal{M}_n(\mathbb{R})} f(\Delta)=\max(d(\Delta,\mathcal{C}_1),d(\Delta,\mathcal{C}_2))$$

avec $d(\Delta,\mathcal{C}_i)$ la distance de $\Delta$ à $\mathcal{C}_i$.

On a donc $$f(\Delta) = \max_{i \in{} \{1,2\}}(\|\Delta{}-\pi_{C_i}(\Delta)\|_F$$

**Question 2 :** On choisit de munir $\mathcal{M}_n(\mathbb{R})$ du produit scalaire $<X,Y>=tr(XY^T)$. On a alors $d(\Delta,\mathcal{C}_i)=\Vert \Delta - \Pi_{\mathcal{C}_i}(\Delta)\Vert_F$, avec $\Vert \Vert_F$ la norme de Frobenius, et $\Pi_{\mathcal{C}_i}(\Delta)$ le projeté de $\Delta$ sur $\mathcal{C}_i$. Donner l'expression analytique des $\Pi_{\mathcal{C}_i}(\Delta)$.

$$\Pi_{C_1}(\Delta) = \frac12 (\Delta + \Delta^T)$$

$$\Pi_{C_2}(\Delta) = \Delta \left( I_n - \frac{\tilde x \tilde x^T}{\|\tilde x\|_F^2} \right) + \frac{(b-A\tilde x)\tilde x^T}{\|\tilde x\|_F^2}$$

In [38]:
function π_Ci(i, Δ, x_tilde, A, b)
    if i == 1
        return 0.5 * (Δ + transpose(Δ))
    else
        squared_norm_x_tilde = norm(x_tilde)^2
        return Δ * (I - x_tilde*transpose(x_tilde)/squared_norm_x_tilde) + (b-A*x_tilde)*transpose(x_tilde)/squared_norm_x_tilde
    end
end

function f(Δ, x_tilde, A, b)
    f_computations = [norm(), norm(Δ-π_Ci(2,Δ,x_tilde,A,b))]
    return Dict('f'=> max(f_computations), 'i' => argmax(f_computations))
end

f (generic function with 1 method)

**Question 3 :** Proposer un sous-gradient de $f$ en $\Delta$, $\forall \Delta \in \mathcal{C}$.

$$g(\Delta)=\frac{\Delta{}-\pi{}_{C_i}(\Delta)}{\|\Delta-\pi{}_{C_i}(\Delta)\|_F}, i \text{  tel que  } f(\Delta)=f_i(\Delta)$$

In [44]:
function sous_grad_f(Δ, x_tilde, A, b)
    f1_unormed = Δ-π_Ci(1,Δ,x_tilde,A,b)
    f2_unormed = Δ-π_Ci(2,Δ,x_tilde,A,b)
    unormed_f_choices = [f1_unormed, f2_unormed]
    f_choices = [norm(f1_unormed), norm(f2_unormed)]
    i = argmax(f_choices)
    return unormed_f_choices[i] / f_choices[i], f_choices[i] #return sous_grad de f et f
end

sous_grad_f (generic function with 1 method)

**Qestion 4 :** Résoudre le problème $ (\mathcal{P})$ par l'algorithme du sous-gradient.

In [61]:
function algo_sous_gradient(x_tilde, A, b, choix_pas = "polyak")
    f_computations_list = []
    f_star = 0  # intersection non-vide donc fstar nulle
    Δ_k = zeros(size(A))
    i=0
    while i <= 1000
        i = i + 1
        sous_grad_f_k, f = sous_grad_f(Δ_k, x_tilde, A, b)
        append!(f_computations_list, f)
        if choix_pas == "polyak"
            step = (f-f_star)/norm(sous_grad_f_k)^2;
        elseif choix_pas == "constant"
            step = 0.01
        elseif choix_pas == "carreint"
            step = 1/(i)
        end
        Δ_k = Δ_k - step*sous_grad_f_k
    end
    return Δ_k, f_computations_list
end

algo_sous_gradient (generic function with 2 methods)

In [62]:
Random.seed!(5)
A = 2*randn(3,3)
A = A+transpose(A)
print("A = ") ; show(stdout, "text/plain", A) ; println()
xtilde = 2*randn(3,1)
print("\nxtilde = ") ; show(stdout, "text/plain", xtilde) ; println()
dt = randn(3,3)/10
print("\ndt = ") ; show(stdout, "text/plain", dt) ; println()
b = (A+dt)*xtilde
print("\nb = ") ; show(stdout, "text/plain", b) ; println("\n")

# polyak step
Δ, f_computations_list = algo_sous_gradient(xtilde, A, b, "polyak")
show(stdout, "text/plain", Δ)  ; println("\n")
plot(f_computations_list, label="polyak")

# contant step 
Δ, f_computations_list = algo_sous_gradient(xtilde, A, b, "constant")
show(stdout, "text/plain", Δ)  ; println("\n")
plot!(f_computations_list, label="constant")

# carreint step
Δ, f_computations_list = algo_sous_gradient(xtilde, A, b, "carreint")
show(stdout, "text/plain", Δ)  ; println("\n")
plot!(f_computations_list, label="carreint")

A = 3×3 Array{Float64,2}:
  5.85723   2.78998  -1.62452
  2.78998  -6.29835  -3.85572
 -1.62452  -3.85572  -0.258518

xtilde = 3×1 Array{Float64,2}:
 0.8150467918489965
 2.8243499155393783
 1.3241022925246149

dt = 3×3 Array{Float64,2}:
 -0.0311238    0.0213278   0.0777298
  0.00983044  -0.0949088   0.00574745
  0.0868369   -0.0418395  -0.0520668

b = 3×1 Array{Float64,2}:
  10.640551109538936
 -20.872573042772764
 -12.672600360490229

3×3 Array{Float64,2}:
 0.026249    0.0337258   0.0159688
 0.0337258  -0.0814616  -0.0376445
 0.0159688  -0.0376445  -0.0173923

3×3 Array{Float64,2}:
 0.0274663   0.0374975   0.0177383
 0.0338808  -0.0824714  -0.0381136
 0.0160527  -0.0380791  -0.0175941

3×3 Array{Float64,2}:
 0.0263718   0.0338304   0.0160187
 0.0340192  -0.0815565  -0.0376859
 0.0161067  -0.0376877  -0.0174112



<!DOCTYPE html>
 
 
 Plots.jl